In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [2]:
x = tf.constant([[1,0],[0,1],[0,0],[1,1]])
y = tf.constant([[1],[1],[0],[0]])

model = tf.keras.Sequential([
    tf.keras.layers.Dense(2,activation='relu'),
    tf.keras.layers.Dense(10,activation='sigmoid'),
    tf.keras.layers.Dense(10,activation='sigmoid'),
    tf.keras.layers.Dense(2,activation='softmax')
])

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(2),
    tf.keras.layers.Dense(128,activation="linear"),
    #tf.keras.layers.Dense(32,activation="linear"),
    tf.keras.layers.Dense(1,activation="sigmoid"),
])

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x,y,epochs=200)

Epoch 1/200
1/1 [==============================] - 1s 524ms/step - loss: 0.6970 - accuracy: 0.7500
Epoch 2/200
1/1 [==============================] - 0s 7ms/step - loss: 0.6956 - accuracy: 0.7500
Epoch 3/200
1/1 [==============================] - 0s 7ms/step - loss: 0.6946 - accuracy: 0.7500
Epoch 4/200
1/1 [==============================] - 0s 8ms/step - loss: 0.6940 - accuracy: 0.7500
Epoch 5/200
1/1 [==============================] - 0s 7ms/step - loss: 0.6937 - accuracy: 0.5000
Epoch 6/200
1/1 [==============================] - 0s 11ms/step - loss: 0.6936 - accuracy: 0.5000
Epoch 7/200
1/1 [==============================] - 0s 9ms/step - loss: 0.6936 - accuracy: 0.5000
Epoch 8/200
1/1 [==============================] - 0s 9ms/step - loss: 0.6937 - accuracy: 0.2500
Epoch 9/200
1/1 [==============================] - 0s 6ms/step - loss: 0.6937 - accuracy: 0.5000
Epoch 10/200
1/1 [==============================] - 0s 8ms/step - loss: 0.6938 - accuracy: 0.5000
Epoch 11/200
1/1 [========

In [9]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=['binary_accuracy'])

model.fit(x,y,epochs=10000)

Epoch 1/10000
1/1 [==============================] - 0s 458ms/step - loss: 0.2522 - binary_accuracy: 0.5000
Epoch 2/10000
1/1 [==============================] - 0s 6ms/step - loss: 0.2513 - binary_accuracy: 0.2500
Epoch 3/10000
1/1 [==============================] - 0s 6ms/step - loss: 0.2507 - binary_accuracy: 0.5000
Epoch 4/10000
1/1 [==============================] - 0s 7ms/step - loss: 0.2503 - binary_accuracy: 0.5000
Epoch 5/10000
1/1 [==============================] - 0s 7ms/step - loss: 0.2502 - binary_accuracy: 0.5000
Epoch 6/10000
1/1 [==============================] - 0s 9ms/step - loss: 0.2502 - binary_accuracy: 0.2500
Epoch 7/10000
1/1 [==============================] - 0s 10ms/step - loss: 0.2503 - binary_accuracy: 0.5000
Epoch 8/10000
1/1 [==============================] - 0s 8ms/step - loss: 0.2504 - binary_accuracy: 0.5000
Epoch 9/10000
1/1 [==============================] - 0s 8ms/step - loss: 0.2505 - binary_accuracy: 0.5000
Epoch 10/10000
1/1 [=======================

In [5]:
model.predict([[0,1],[1,0],[0,0],[1,1]])

array([[0.01433893, 0.98566103],
       [0.01409426, 0.98590577],
       [0.8992424 , 0.10075759],
       [0.9863323 , 0.01366775]], dtype=float32)